<a href="https://colab.research.google.com/github/cjweems/image-filter-project/blob/feedback/Image_Filter_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive

In [14]:
cd content/drive/MyDrive/'AP CSP'/'Image Filter Project'

/content/drive/MyDrive/AP CSP/Image Filter Project


In [20]:
import cv2
import os
import matplotlib.pyplot as plt

# Get an image from google drive
directory = os.getcwd()
imagePath = os.path.join(directory, 'person.png')

# Read the image
img = cv2.imread(imagePath)

# Conver the image to grayscale
gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Load the pre-trained classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") #Note the change

# Preform the face detection
face = face_cascade.detectMultiScale(gray_img,1.1,4)

# Draw the bounding box
for (x, y, w, h) in face:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)

# Display the image
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,10))
plt.imshow(img_rgb)
plt.axis('off')

error: ignored

In [2]:
cd drive/MyDrive/'AP CSP'/'Image Filter Project'

[Errno 2] No such file or directory: 'drive/MyDrive/AP CSP/Image Filter Project'
/content
